# 벡터자기회귀(VAR) 모형

In [1]:
# # VAR모형의 가장 큰 특징
# (1) 충격반응분석(impulse response analysis)을 통하여 어떠한 한 변수의 변화가 내생변수에 미치는 동태적 반응을 파악할 수 있다.
# (2) 분산분해(variance decomposition)를 통하여 각 내생변수의 변동이 전체변동에 기여한 부분의 상대적 크기를 분석할 수 있다.
# (3) 경제이론 보다는 실제 자료에서 도출된 결과를 분석한다.
# (4) 그러나 VAR모형은 사용되는 변수 및 표본기간, 시차길이를 따라서 결과가 달라질 수 있다는 약점이 있다.
# (VAR 모형의 예시: 환율과 수출 물가에 의한 상품수출모형을 설정하여 모형 설정과정과 해석방법 등을 설명)

# # 설명
# 벡터자기회귀모형은 일변량 자기회귀모형을 다변량 자기회귀모형으로 확정시킨 모형으로
# 예측 및 내생변수의 변화에 따른 효과분석 등과 관련하여 자주 활용되고 있다.
# 전통적인 회귀모형에 의한 구조방정식모형은 변수간의 인과관계를 통하여 종속변수 Y를 몇 개의 설명 변수 {X1, X2, …}에 의해서 설명하고 있다.
# 그러나 회귀모형에서는 설명변수의 영향이 시간 t가 변하더라도 항상 일정하다는 가정을 하고 있어
# 구조적 변화가 급속히 진행되어 설명변수의 영향이 변한 경우 이를 적절히 반영하지 못한다는 약점이 있다.
# 또한 구조모형(structuremodel)은 경제이론에 의해서 모형을 구축하고 있어
# 변수선택 및 모형의 내․외생변수의 선정이 모형 설계자의 주관에 의해서 결정된다는 단점이 있다.
# 따라서 이러한 시간에 대한 경직성과 주관성을 극복할 수 있는 방법이 Box and Jenkins(1976)의 ARIMA모형이라고 할 수 있다.
# ARIMA모형은 현재의 관측치 Zt는 과거의 어떠한 규칙성에 의해서 재현되며,
# 이러한 규칙성은 미래에도 유지된다고 가정하고 미래를 예측하고자 했다.
# 이러한 방법은 모형 설정이 용이한 반면 변수들 사이의 상호작용을 무시하고 있어 일변량분석이라는 한계에 부딪치게 된다.
# 이들 회귀모형과 시계열분석의 한계를 보완한 모형이 Sims(1980)의 VAR모형이라 할 수 있다.
# [출처 : 통계청『통계분석연구』제2권 제1호(‘97.봄)23-56 / 벡터자기회귀(VAR)모형의 이해 / Vector Autoregressive Model: VAR / 문권순]
# => 모든 변수는 시스템 안의 다른 모든 변수에 영향을 준다는 가정을 하기 때문에, 추정한 계수를 해석하는 것이 어렵다.

# # 정리
# VAR 모형은 종속변수와 독립변수가 서로 영향을 주고 받는다는 가정하에 사용되기 때문에
# train data(종속변수 + 독립변수)를 이용하여 모델을 학습하고 해당 train data(종속변수 + 독립변수)를 이용하여 예측값을 생성함
# 즉, 독립변수와 종속변수가 서로 영향을 주는 관계이기 때문에 독립변수와 종속변수가 구분되어 있다고 할 수 없음
# => 독립변수가 종속변수에 영향을 주고, 종속변수가 독립변수에 영향을 주는 관계일 때 사용하는 모형

In [2]:
# # 차분(differencing)
# - 연이은 관측값들의 차이를 계산하는 것
# - 로그 같은 변환은 시계열의 분산 변화를 일정하게 만드는데 도움이 될 수 있음
# - 차분(differencing)은 시계열의 수준에서 나타나는 변화를 제거하여 시계열의 평균 변화를 일정하게 만드는데 도움이 될 수 있음
# - 결과적으로 추세나 계절성이 제거(또는 감소)됨

# # 정상성(Stationary)
# - 정상성(stationarity)을 나타내는 시계열은 '시계열의 특징'이 '해당 시계열이 관측된 시간에 무관'
# - 즉, {Yt}가 정상성을 나타내는 시계열이라면, 모든 s에 대해 (Yt, ..., Yt+s)의 분포에 t가 무관함
# - 이 말은 추세나 계절성이 있는 시계열은 정상성을 나타내는 시계열이 아니라는 점(추세와 계절성은 서로 다른 시간에 시계열의 값에 영향을 주기 때문)
# - 그래서 정상성을 나타내는 시계열을 '백색잡음(white noise)'라고 하기도 함(언제 관찰하든, 시간이 어떻든 똑같이 보이기 때문)
# - 주기(cycle)이 있긴 하지만 이 주기가 불규칙적이기 때문에 정상성이 있다고 본다고 함

----------
# 0. 환경설정

In [3]:
# 라이브러리 호출
import numpy as np
import pandas as pd
import time
import glob
import pickle
import itertools
import copy

import statsmodels.api as sm
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

# 데이터프레임 출력 옵션
pd.set_option('display.max_columns', 100)

#지수표현
pd.options.display.float_format = '{:.5f}'.format

# # 그래프 폰트
# font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
# rc('font', family=font_name)
# plt.rc('font', family='Malgun Gothic')
# plt.rcParams["figure.figsize"] = (8, 4)
# plt.rcParams['axes.unicode_minus'] = False

----------
# 1. 입력값 기입

## 1-1. 기본 정보

In [4]:
data_folder = 'data'    # 원본데이터 위치(폴더명)
save_folder = 'result'  # 결과 저장 위치(폴더명)
file_nm = 'ts_data(jena_climate_2009_2016)'  # 원본파일명(csv 파일)
sv_file_nm = 'result_ts_data(jena_climate_2009_2016)'  # 결과파일명(csv 파일)
file_encode = 'utf-8'     # 원본데이터의 encoding
sv_file_encode = 'utf-8'  # 결과데이터의 encoding

## 1-2. 표준화 정보

In [5]:
## 독립변수(x)에 대하여 표준화 진행 시 원하는 표준화 기법 선택 (입력 안하면 표준화 진행 X)
# 1 : 표준화1(StandardScaler) : 평균 = 0 / 표준편차 = 1
# 2 : 표준화2(Normalization) : MinMaxScaler : 최소값 0 ~ 최대값 1 : 이상값에 영향 받음
# 3 : 표준화3(RobustScaler) : 중앙값 = 0 / IQR(1분위(25%) ~ 3분위(75%)) = 1 
select_scaler = 1  # (1, 2, 3) 중 택 1

## 1-3. 상관분석 정보

In [6]:
## 상관계수 값 설정 (y와 x변수간의 단일 상관계수 값이 'corr_stand_val' 이상인 x변수만 입력변수로 사용)
# 입력변수 : 모델링에 이용되는 변수
corr_stand_val = 0.5

## 1-4. test set 비율 정보

In [7]:
# 전체 데이터셋 중 test_per 비율만큼 오차율 계산
test_per = 0.2

----
# 2. 데이터 호출

In [8]:
## 아래의 데이터를 저장하여 사용함
# import tensorflow as tf
# import matplotlib as mpl
# import matplotlib.pyplot as plt
# import numpy as np
# import os
# import pandas as pd

# mpl.rcParams['figure.figsize'] = (8, 6)
# mpl.rcParams['axes.grid'] = False

# zip_path = tf.keras.utils.get_file(
#   origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
#   fname='jena_climate_2009_2016.csv.zip',
#   extract=True)
# csv_path, _ = os.path.splitext(zip_path)
# df = pd.read_csv(csv_path)
# df.to_csv('ts_data(jena_climate_2009_2016).csv', index=False, encoding = 'utf-8')

In [9]:
# ## 컬럼 설명
# Index   Features           Format                  Description
# 1       Date Time          01.01.2009 00:10:00     Date-time reference
# 2       p (mbar)           996.52                  The pascal SI derived unit of pressure used to quantify internal pressure. Meteorological reports typically state atmospheric pressure in millibars.
# 3       T (degC)           -8.02                   Temperature in Celsius
# 4       Tpot (K)           265.4                   Temperature in Kelvin
# 5       Tdew (degC)        -8.9                    Temperature in Celsius relative to humidity. Dew Point is a measure of the absolute amount of water in the air, the DP is the temperature at which the air cannot hold all the moisture in it and water condenses.
# 6       rh (%)             93.3                    Relative Humidity is a measure of how saturated the air is with water vapor, the %RH determines the amount of water contained within collection objects.
# 7       VPmax (mbar)       3.33                    Saturation vapor pressure
# 8       VPact (mbar)       3.11                    Vapor pressure
# 9       VPdef (mbar)       0.22                    Vapor pressure deficit
# 10      sh (g/kg)          1.94                    Specific humidity
# 11      H2OC (mmol/mol)    3.12                    Water vapor concentration
# 12      rho (g/m ** 3)     1307.75                 Airtight
# 13      wv (m/s)           1.03                    Wind speed
# 14      max. wv (m/s)      1.75                    Maximum wind speed
# 15      wd (deg)           152.3                   Wind direction in degrees

In [10]:
# 데이터 호출 및 필요 컬럼 정리
data = pd.read_csv(data_folder + '/' + file_nm + '.csv', encoding = file_encode)
data.columns = list(pd.Series(list(data.columns)).map(lambda x : x.replace(' ','')))

# (날짜변수/종속변수/독립변수) 컬럼명 정의 : x_colnm
tm_colnm = 'DateTime'
y_colnm = 'T(degC)'
x_colnm = list(set(data.columns).difference({y_colnm}).difference({tm_colnm}))

# Y(t+1)은 X(t)들에 의해 영향을 받음
data = pd.concat([data[y_colnm].shift(-1), data[x_colnm]], axis = 1)

# 데이터 형 변환(str -> float)
data[[y_colnm] + x_colnm] = data[[y_colnm] + x_colnm].astype('float')

In [11]:
# # 임시(데이터 건수 줄여서 테스트 진행)
data = data.iloc[:1000]

------
# 3. 모델링

## 3-1. 데이터 전처리

In [12]:
## 표준화 수행

# StandardScaler : 평균 0, 표준편차 1
if select_scaler == 1:
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    data[x_colnm] = pd.DataFrame(scaler.fit_transform(data[x_colnm]), columns = list(data[x_colnm].columns))

# Normalization : MinMaxScaler : 최소값 0 ~ 최대값 1
elif select_scaler == 2:
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    data[x_colnm] = pd.DataFrame(scaler.fit_transform(data[x_colnm]), columns = list(data[x_colnm].columns))

# RobustScaler : 중앙값 0, IQR(1분위(25%) ~ 3분위(75%)) 1 : 이상치(outlier) 영향 최소화 / 더 넓게 분포
elif select_scaler == 3:
    from sklearn.preprocessing import RobustScaler
    scaler = RobustScaler()
    data[x_colnm] = pd.DataFrame(scaler.fit_transform(data[x_colnm]), columns = list(data[x_colnm].columns))

# 표준화 수행 X
else:
    pass

In [13]:
# inf / -inf 값을 null 처리
data = data.replace([np.inf, -np.inf], np.nan)

In [14]:
## 독립변수 선택

# 1. 상관분석 : 상관분석은 train 데이터셋에 대해서만 진행 (test 데이터셋 이용 X)
corr = data.corr(method = 'pearson')  # default는 'pearson'
corr = corr.reset_index().rename(columns = {'index':'COLNM'})
corr = corr.loc[corr['COLNM'] != y_colnm,]
corr = corr[corr[y_colnm] >= corr_stand_val]

# 2. 상관분석 결과로 선택된 독립변수 목록
mdl_x_colnm = list(corr['COLNM'])

# 3. 독립변수 목록 중 null값이 없는 독립변수만 선택
na_col = []
for col in data.columns:
    if len(data.loc[data[col].isna(),]) != 0:
        na_col.append(col)

mdl_x_colnm = list(set(mdl_x_colnm).difference(set(na_col)))  # 모델에 사용할 독립변수들(상관분석 결과 - null값이 있는 변수 제거)

if len(mdl_x_colnm) == 0:  # 모델에 사용할 독립변수의 수가 0이면 => 기본 독립변수 중 null값이 없는 변수를 선택 
    mdl_x_colnm = x_colnm
    mdl_x_colnm = list(set(mdl_x_colnm).difference(set(na_col)))

In [15]:
# 전처리 된 데이터셋
mody_data = data[[y_colnm] + mdl_x_colnm]

In [16]:
# train data와 test data 분리
test_cnt = int(round((len(mody_data) * test_per),0))  # test set 건수 = 전체 데이터셋 * test_per(비율)
train_data = mody_data[:len(mody_data) - test_cnt].reset_index(drop=True)
test_data = mody_data[-test_cnt:].reset_index(drop=True)

## 3-2. 정상성 확인 (ADF-Test)

In [17]:
## 정상성 확인 (ADF-Test)
# 귀무가설 : 정상성이 없다 = 시계열 흐름이 불규칙하다
# p-value가 0.05보다 크면 귀무가설을 기각할 수 없음 = 정상성이 없다 = 시계열 흐름이 불규칙하다
ADF_test_statistic = []
p_value = []
for i in train_data.columns[1:]:
    adfuller_test = adfuller(train_data[i], autolag= "AIC")
    ADF_test_statistic.append(adfuller_test[0])
    p_value.append(adfuller_test[1])

ADF_col_nm = pd.DataFrame(train_data.columns[1:]).rename(columns = {0:'col_nm'})
ADF_test_statistic = pd.DataFrame(ADF_test_statistic).rename(columns = {0:'ADF_test_statistic'})
p_value = pd.DataFrame(p_value).rename(columns = {0:'p_value'})

ADF_test = pd.concat([ADF_col_nm, ADF_test_statistic, p_value], axis = 1)  # 전체 ADF_test 결과 출력

# # ADF_test 결과, 정상성을 만족하지 않는 컬럼 목록 : 차분 필요
diff_list = list(ADF_test.loc[ADF_test['p_value'] >= 0.05,'col_nm'])

In [18]:
# ADF_test 결과 확인
ADF_test

,col_nm,ADF_test_statistic,p_value
0,Tpot(K),-1.77611,0.39239
1,sh(g/kg),-0.94497,0.77280
2,VPmax(mbar),-1.46610,0.55021
3,H2OC(mmol/mol),-0.94971,0.77116
4,Tdew(degC),-0.94636,0.77232
5,VPact(mbar),-0.93889,0.77490


In [19]:
# 정상성을 만족하지 않는 컬럼의 경우, 차분한 후 다시 정상성 확인
if len(diff_list) != 0:

    # 차분 데이터 생성
    diff_data = train_data[diff_list].diff(1).dropna()

    # 차분 데이터의 정상성 확인 (ADF-Test)
    ADF_test_statistic = []
    p_value = []
    for i in diff_data.columns:
        adfuller_test = adfuller(diff_data[i], autolag= "AIC")
        ADF_test_statistic.append(adfuller_test[0])
        p_value.append(adfuller_test[1])

    ADF_col_nm = pd.DataFrame(diff_data.columns[1:]).rename(columns = {0:'col_nm'})
    ADF_test_statistic = pd.DataFrame(ADF_test_statistic).rename(columns = {0:'ADF_test_statistic'})
    p_value = pd.DataFrame(p_value).rename(columns = {0:'p_value'})

    ADF_test = pd.concat([ADF_col_nm, ADF_test_statistic, p_value], axis = 1)

    # # ADF_test 결과, 정상성을 만족하지 않는 컬럼 목록 : 차분 필요
    diff_list_1 = list(ADF_test.loc[ADF_test['p_value'] >= 0.05,'col_nm'])

In [20]:
# 1차 차분 외 추가로 차분해야하는 컬럼 목록
diff_list_1

[]

In [21]:
# '1차 차분' 진행 후 차분으로 인해 생성된 null값 제거 (여기서는 1차 차분만 진행 - 필요에 따라 추가 차분 코드 작성)
train_data[diff_list] = train_data[diff_list].diff(1)
train_data = train_data.dropna()

## 3-3. VAR 모형 생성

In [22]:
## VAR 모형의 최적 순서 탐색 : (1) AIC 값은 낮을수록 좋음 / (2) Grid-Search를 수행하여 최소 AIC를 만족하는 최적의 p를 탐색
model = VAR(train_data)

fit_param = []
result_aic = []
for p in range(1,len(train_data) + 1):
    try :
        fitted_model = model.fit(p)  # 모델 생성
        fit_param.append(p)
        result_aic.append(fitted_model.aic)
    except :
        pass

fit_param = pd.DataFrame(fit_param, columns = ['fit_param'])
result_aic = pd.DataFrame(result_aic, columns = ['result_aic'])
fit_param_df = pd.concat([fit_param,result_aic], axis = 1)

# 최적의 p 값 도출 : fit_param_val
fit_param_val = list(fit_param_df.loc[fit_param_df['result_aic'] == fit_param_df['result_aic'].min(), 'fit_param'])[0]

# # 최적 p 값 그래프로 확인 (필요시 확인)
# plt.plot(fit_param_df['fit_param'], fit_param_df['result_aic'])
# plt.plot(fit_param_df['fit_param'][0:200], fit_param_df['result_aic'][0:200])

In [23]:
## 모델 생성(model fitting)
fitted_model = model.fit(fit_param_val)
# fitted_model.summary()  # 모델 fitting 결과 (필요시 확인)

In [24]:
# lagged_values 표현식 : train_data[-fit_param_val:].values 또는 np.array(train_data[-fit_param_val:])
lagged_values = np.array(train_data[-fit_param_val:])  # 예측을 위해 사용할 데이터
predict = pd.DataFrame(fitted_model.forecast(lagged_values, steps = test_cnt), columns = list(train_data.columns))

In [25]:
# 결과 확인
df_pred = predict[[y_colnm]].rename(columns = {y_colnm:'pred'})    # pred 값
df_real = test_data[[y_colnm]].rename(columns = {y_colnm:'real'})  # real 값

result = pd.concat([df_pred, df_real], axis = 1)
result['error_rate'] = ((result['pred'] - result['real'])/result['real']) * 100

In [26]:
result

,pred,real,error_rate
0,-10.30600,-11.07000,-6.90153
1,-9.82224,-11.33000,-13.30764
2,-9.43270,-11.90000,-20.73360
3,-9.01151,-12.43000,-27.50197
4,-8.34349,-12.47000,-33.09153
...,...,...,...
195,-17.35102,-9.30000,86.57007
196,1042.24079,-9.89000,-10638.32949
197,-605.37363,-10.44000,5698.59797
198,-1217.22466,-10.83000,11139.37821


In [27]:
# 결과 저장
result.to_csv(save_folder + '/' + sv_file_nm + '.csv', index=False, encoding = sv_file_encode)

------------